Programmieren 3 - Unterstützung zum Thema "Schiefer Wurf mit Luftwiderstand"

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023/2024

# Prototypen-Entwicklung mit Jupyter - Schiefer Wurf

**Aufgabe:** Berechnen Sie die Bahnkurven für den schiefen Wurf im Vakuum auf der Erde für verschiedene Startwinkel und Startgeschwindigkeiten. Stellen Sie das Ergebnis grafisch dar.

**Problem verstehen:** Gibt es Fragen?

**Recherche / Theorie**: </br>
Es wirkt die Gewichtskraft in negative z-Richtung
$$ F_z = -m g, \; g = 9.81 \frac{m}{s^2} $$
Daraus ergibt sich die konstante Beschleunigung in $z$ -Richtung
$$a_z = -g$$
also
$$\vec{a} = \left( \begin{array}{c} 0 \\ 0 \\ -g \end{array}\right)$$

Die Bewegung findet in einer Ebene statt. Entscheidung: Wir betrachten die $xz$-Ebene.
Vorgegeben ist eine Anfangsgeschwindigkeit $|v_0|$, ein Startwinkel $\alpha$ und ein Startpunkt $\vec{x}_0$
Daraus folgt:
$$\vec{v_0} = \left( \begin{array}{c} v_0 \cos(\alpha) \\ 0 \\ v_0 \sin(\alpha) \end{array}\right)$$
Iteration:
$$\vec{x}_{n+1} = \vec{x}_n + \vec{v} \, \Delta t + \vec{a} \, \frac{\Delta t^2}{2}$$
$$\vec{v}_{n+1} = \vec{v}_n +  \vec{a} \, \Delta t$$

**Teilaufgaben:**
    
* Initialisierung des Systems (Anfangsbedingungen)
* Berechnung und Speicherung der Bahnkurve
* Visualiseriung der Ergebnisse

**Ansätze und Entscheidungen:** (Algorithmen und Datenstrukturen)

* Verwendung von *numpy*-Arrays für die Vektoren $\vec{x}, \vec{v}, \vec{a}$
* Schrittweise Simulation unter Verwendung der oben gegebenen Formeln
* Speichern der Position als Funktion der Zeit in einem *numpy*-array
* Anzeige mit *matplotlib*


In [ ]:
# Initialisierung
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib ipympl
#
# Start
x0 = np.array([20, 0, 10], np.float32)
v0_abs = 20
alpha = 45
#alpha = 0
alpha_rad = math.radians(alpha)
g = 9.81
a = np.array([0, 0, -g], np.float32)
delta_t = 0.01
x = x0.copy()
v = np.array([v0_abs*math.cos(alpha_rad), 0, 
              v0_abs*math.sin(alpha_rad)],
              np.float32)

In [ ]:
#
# Liste fuer die Zwischenpositionen
pos_list = []
#
# Berechnung
while x[2] >= 0.0:
    x = x + v * delta_t + a * delta_t**2 / 2
    v = v + a * delta_t
    pos_list.append(x)
#
# Anzeige
ax = plt.figure().add_subplot(111)
ax.plot([x[0] for x in pos_list], [x[2] for x in pos_list])

# Gruppenarbeit

Erweitern Sie die oben gegebene Simulation des schiefen Wurfs, so dass der Luftwiderstand berücksichtigt wird. Es soll eine Kugel der Masse 10 kg und variablem Radius $r$ (Einheit: m) bei $20^{\circ}$ C auf Meereshöhe betrachtet werden. Die Parameter $\alpha$, $|v_0|$ und $r$ sollen über Schieberegler im Notebook (Stichwort *interact*) modifizierbar sein. Wie ändert sich die Kurve mit wachsendem $r$?

Bitte teilen Sie die Aufgaben innerhalb des Teams auf.

**Hilfestellung:**
\begin{equation}
\vec{F}_{\rm L} = - \frac{1}{2} \vec{v} \cdot |\vec{v}| \cdot A \cdot c_W \cdot \rho_{\rm Luft},\; A = \pi r^2
\end{equation}

Dabei bezeichnet $\rho_{\rm Luft}$ die Luftdichte und $c_W$ den Strömungswiderstandskoeffizienten. Wir setzen für die Kugel [$c_W = 0.5$](https://www.code-knacker.de/cw-wert.htm).

**Nach 10 Minuten treffen wir uns wieder kurz im Plenum, um offene Fragen zu klären.**

# Vorgehensweise  (Vorschlag)

## Zerlegung in Teilaufgaben

1. Teilprobleme in der Formel
    1. Finden der fehlenden Konstante $\rho_{\rm Luft}$
    1. Berechnung der Länge eines Vektors
    1. Eigentliche Bestimmung von $\vec{F}_{\rm L}$
1. Integration von $\vec{F}_{\rm L}$ in die vorgegebene Lösung
1. Refaktorisierung des Codes
    1. Strukturierung (Funktionen, Konstanten, etc.)
    1. Definition einer Schnittstelle für eine interaktive Anwendung
1. Realisierung der interaktiven Anwendung

## Recherche

* https://de.wikipedia.org/wiki/Luftdichte: $\rho_{\rm Luft} = 1.2 \frac{\rm kg}{m^3}$ 
* Berechnung der Länge eines Vektors, 3 Möglichkeiten:

        l_v = math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
        l_v = math.sqrt(np.dot(v, v))
        l_v = np.linalg.norm(v) 

## Algorithmen und Datenstrukturen

Es soll eine Funktion erstellt werden, die $\vec{F}_{\rm L}$ berechnet.
* Eingabegrößen: Geschwindigkeit $\vec{v}$ (numpy-arrray, x/y/z), Radius r (float)
* Ausgabegröße: $\vec{F}_{\rm L}$ (numpy-array, x/y/z)

Außerdem soll die Berechnung der entsprechenden Beschleunigung als Funktion zur Verfügung gestellt werden.
* Eingabegrößen: Geschwindigkeit $\vec{v}$ (numpy-arrray, x/y/z), Radius r (float), Masse m (float)
* Ausgabegröße: $\vec{a}_{\rm L}$ (numpy-array, x/y/z)

## Implementierung und vertrauensbildende Maßnahme

In [ ]:
# Konstanten
CW_KUGEL = 0.5
RHO_LUFT = 1.2


def luftwiderstand_kraft(v: np.ndarray, r: float) -> np.ndarray:
    """Berechnung der aus dem Luftwiderstand resultierenden Kraft

    Args:
        v (np.array, x/y/z): Geschwindigkeits-Vektor in m/s
        r (float): Radius in m

    Returns:
        Kraft F_L (np.array, x/y/z): Kraft in kg m / s**2
    """
    l_v = np.linalg.norm(v)
    A = np.pi * r**2
    F_L = -1 / 2 * v * l_v * A * CW_KUGEL * RHO_LUFT
    return F_L


def luftwiderstand_beschleunigung(
    v: np.ndarray, r: float, m: float
) -> np.ndarray:
    """Berechnung der aus dem Luftwiderstand resultierenden Beschleunigung

    Args:
        v (np.array, x/y/z): Geschwindigkeits-Vektor in m/s
        r (float): Radius in m
        m (float): Masse in kg

    Returns:
        Beschleunigung a_L (np.array, x/y/z): Beschleunigung in m / s**2
    """
    a_L = luftwiderstand_kraft(v, r) / m
    return a_L

### Plausibilität

**Hinweis**: Dieser Untersuchung war eigentlich nicht gefragt und ist als "Fingerübung" für *numpy* gedacht ... 

**Ziel:** Schaffen von Vertrauen in die oben implementierten Funktionen.

**Frage:** Welche Geschwindigkeit kann ein Fallschirmspringer im freien Fall (negative z-Richtung) maximal erreichen?

**Annahmen/Näherung:** Wir nehmen an, ein Fallschirmspringer hat ein effektive Fläche von  $(1.8 \cdot 0.5)m^2$ und wiegt 80 kg. Die Luftdichte sei unabhängig von der Höhe.

**Idee:** Der Fallschirmspringer wird so lange immer schneller, bis die Erdbeschleunigung durch die aus dem Luftwidertand resultierende Beschleunigung aufgehoben wird. Wir suchen also die Geschwindigkeit $|\vec{v}_{\rm max}|$, für die gilt:
$\vec{a}_{\rm L}(\vec{v}_{\rm max}) = \left( \begin{array}{rrr} 0\\ 0\\ g \end{array} \right)$

**Umsetzung:** Anstatt die Gleichungen einfach aufzulösen, erzeugen wir eine grafische Lösung mit unserer Implementierung.

In [ ]:
# Anzahl der Punkte
n_punkte = 100
# Geschwindigkeits-Bereich, der betrachtet werden soll in m/s
geschwindigeits_bereich = (0, 100)
# Feld für die Geschwindigkeiten
v_array = np.zeros((n_punkte, 3), dtype=np.float32)
# Belegen der z-Komponenten mit den Test-Geschwindikgeiten
schrittweite = (
    geschwindigeits_bereich[1] - geschwindigeits_bereich[0]
) / n_punkte
v_array[:, 2] = np.arange(
    geschwindigeits_bereich[0], geschwindigeits_bereich[1], schrittweite
)
# Feld für die Beschleunigungen
a_L = np.full_like(v_array, fill_value=0)
# Berechnung des Radius aus der effektiven Fläche
A = 1.8 * 0.5
eff_radius = np.sqrt(A / np.pi)
# Speichern der Beschleunigungen im Feld a_L
for index, v in enumerate(v_array):
    a_L[index] = luftwiderstand_beschleunigung(v, r=eff_radius, m=80)
# Plot vergrößern, damit mehr zu erkennen ist ...
ax = plt.figure().add_subplot(111)
# Gitter einzeichnen
ax.grid()
# Wir skalieren die x-Achse mit 3.6, um km/h zu erhalten
x = v_array[:, 2] * 3.6
ax.set_xlabel("v / km/h")
# Die Maximal-Geschwindigkeit entspricht dem Nulldurchgang (wegen +g)
y = a_L[:, 2] + g
ax.set_ylabel("a + g")
# Anzeigen der Kurve
ax.plot(x, y)

Es ergibt sich eine Grenzgeschwindigkeit knapp unterhalb von 200 km/h. Das ist plausibel, siehe [swr.de](https://www.swr.de/wissen/1000-antworten/wissenschaft-und-forschung/1000-antworten-1384.html)

## Integration in die existierende Lösung (Prototyp)

Wir benennen die Variable *a* in der Initialiseriungs-Zelle in a_g um und nennen die Startgeschwindigkeit *v0*.

In [ ]:
# Initialisierung
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
#
# Start
x0 = np.array([20, 0, 10], np.float32)
v0_abs = 20
alpha = 45
#alpha = 0
alpha_rad = math.radians(alpha)
g = 9.81
a_g = np.array([0, 0, -g], np.float32)
delta_t = 0.01
v0 = np.array([v0_abs*math.cos(alpha_rad), 0, 
              v0_abs*math.sin(alpha_rad)],
              np.float32)

Wir vergleichen Bahnkurven mit und ohne Luftwiderstand

In [ ]:
#
# Liste fuer die Zwischenpositionen ohne Luftwiderstand
pos_list_ohne_FL = []
#
# Berechnung ohne FL
x = x0.copy()
v = v0.copy()
while x[2] >= 0.0:
    x = x + v * delta_t + a_g * delta_t**2 / 2
    v = v + a_g * delta_t
    pos_list_ohne_FL.append(x)
#
# Berechnung mit FL
pos_list_mit_FL = []
x = x0.copy()
v = v0.copy()
while x[2] >= 0.0:
    a = a_g + luftwiderstand_beschleunigung(v, r=1, m=10)
    x = x + v * delta_t + a * delta_t**2 / 2
    v = v + a * delta_t
    pos_list_mit_FL.append(x)
#
# Anzeige
ax = plt.figure().add_subplot(111)
ax.plot([x[0] for x in pos_list_ohne_FL], [x[2] for x in pos_list_ohne_FL])
ax.plot([x[0] for x in pos_list_mit_FL], [x[2] for x in pos_list_mit_FL])

## Refactoring und Interaktion

Diese Schritte sind Ihnen überlassen ...